# Fully Connected Networks

Wir beschäftigen uns in diesen Notebook mit Neuronalen Netzen, die fully connected Layer haben. Zunächst wollen wir eine einfache Funktion mit einer Variablen fitten und anschließend klassifizieren wir Bilder von Ziffern. 

## Eindimensionale Funktion fitten

### Aufgabe 1

Lade zuerst die Datensätze `1d_dataset_train.pt` und `1d_dataset_test.pt` und stelle sie grafisch dar. Die Funktion [`torch.load`](https://pytorch.org/docs/stable/generated/torch.load.html) könnte dabei hilfreich sein.

In [ ]:
import torch
import numpy as np

from matplotlib import pyplot as plt


data_train = torch.load('data/1d_dataset_train.pt')
x_train = data_train['x']
y_train = data_train['y']

data_test = torch.load('data/1d_dataset_test.pt')
x_test = data_test['x']
y_test = data_test['y']

In [ ]:
plt.plot(x_train, y_train, '.b', markersize=0.1)
plt.plot(x_test, y_test, '.r', markersize=0.1)
plt.legend(['Train', 'Test'])
plt.show()

In [ ]:
plt.scatter(x_train, y_train, s=0.1)
plt.show()

### Aufgabe 2

Nun wollen wir uns ein neuronales Netz bauen. In `torch.nn` sind viele Layer implementiert, die wir nutzen wollen. Hier benötigen wir das fully connected Layer [`torch.nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) sowie die Aktivierungsfunktion [`torch.nn.ReLU`](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html). Die beiden genannten Funktionen sind nicht die Layer selbst, sondern sie erzeugen ein Layer, was in folgenden Beispiel veranschaulicht wird.

In [ ]:
# Hiermit erzeugen wir das Layer.
# Es verlangt 2 Feature als Input und gibt 3 Feature zurück
linear_layer = torch.nn.Linear(in_features=2, out_features=3) 

# Um die Anwendung des Layers zu testen, erzeugen wir zunächst einen Beispieldatenpunkt.
x = torch.randn(1, 2) # Batch size: 1, Anzahl der Feature: 2
print('x: ', x)

# Nun können wir den Datenpunkt mit dem Layer verarbeiten und das Ergebnis anzeigen lassen.
out = linear_layer(x)
print('output: ', out)

In [ ]:
# Aktivierungsfunktion
act_fn = torch.nn.ReLU()

print('x: ', x)

out_lin = linear_layer(x)
print('output linear layer: ', out_lin)

out_act = act_fn(out_lin)
print('output linear layer: ', out_act)

Einzelne Layer können mithilfe des Befehls [`torch.nn.Sequential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) zu einem Netzwerk zusammen gefügt werden.

Baue ein neuronales Netz, was aus einem Input Layer, einem Output Layer und zwei hidden Layer mit jeweils 16 hidden Units.

In [ ]:
net = torch.nn.Sequential(
    torch.nn.Linear(1, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 1)    
)

# Teste dein Netzwerk
x = torch.randn(2, 1)
print('Testdatenpunkt x: ', x)
out = net(x)
print('Output: ', out)

### Aufgabe 3

Um das Netzwerk trainieren zu können, benötigen wir eine Loss-Funktion und einen `Optimizer`. Als Loss wollen wir den Mean Squared Error (MSE) verwenden, der in PyTorch [`torch.nn.MSELoss`](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html) heißt.

Ein sehr populärer `Optimizer`, den wir nutzen wollen, ist [`torch.optim.Adam`](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html).

Erzeuge die Loss-Funktion und den `Optimizer` für dein Netzwerk.

In [ ]:
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters())

### Aufgabe 4

Jetzt sind wir bereit für das Training. In jedem Trainingsschritt müssen wir zufällig ein Mini-Batch aus dem Datensatz ziehen, wobei die Funktion [`torch.randint`](https://pytorch.org/docs/stable/generated/torch.randint.html) hilfreich sein könnte. Dieses Mini-Batch wird in das Netzwerk gegeben und aus dem vorhergesagten und den tatsächlichen Werten für `y` wird der Loss ermittelt. Der Backpropagation-Algorithmus wird via `loss.backward()` angewandt und anschließend werden die Parameter des Netzwerks geupdated mit `optimizer.step()`. Nach jedem Schritt müssen die Gradienten mittels `optimizer.zero_grad()` zurückgesetzt werden.

Dieser Trainingsschritt wird mit einer `for`-Schleife mehrfach wiederholt, bis das Netzwerk trainiert ist.

Implementiere den Trainingsalgorithmus und trainiere dein Netzwerk.

In [ ]:
from tqdm import tqdm

max_iter = 1000

for it in tqdm(range(max_iter)):
    # Erzeuge Mini-Batch
    batch_ind = torch.randint(len(x_train), (16,))
    x_batch = x_train[batch_ind, :]
    y_batch = y_train[batch_ind, :]
    
    # Vorhersage des Netzwerks
    y_pred = net(x_batch)
    
    # Loss berechnen
    loss = loss_fn(y_pred, y_batch)
    
    # Backpropagation
    loss.backward()
    
    # Optimizer step
    optimizer.step()
    optimizer.zero_grad()

### Aufgabe 5

Berechne die Vorhersagen des Netzwerkes und bestimme den Loss auf dem Testset.

In [ ]:
with torch.no_grad():
    y_pred_test = net(x_test)
    print(loss_fn(y_pred_test, y_test))

### Aufgabe 6

Erweitere deinen Trainingsalgorithmus, sodass schon während des Trainings die Performance des Netzes in regelmäßigen Abständen bestimmt wird und der Loss gespeichert wird, sodass du ihn nach dem Training plotten kannst.

Passe außerdem deine Netzwerkarchitektur und die Trainingsparameter an, sodass du bessere Ergebnisse bekommst.

In [ ]:
net = torch.nn.Sequential(
    torch.nn.Linear(1, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 1)    
)


loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters())


max_iter = 2000

loss_hist = []

for it in tqdm(range(max_iter)):
    # Erzeuge Mini-Batch
    batch_ind = torch.randint(len(x_train), (64,))
    x_batch = x_train[batch_ind, :]
    y_batch = y_train[batch_ind, :]
    
    # Vorhersage des Netzwerks
    y_pred = net(x_batch)
    
    # Loss berechnen
    loss = loss_fn(y_pred, y_batch)
    
    # Backpropagation
    loss.backward()
    
    # Optimizer step
    optimizer.step()
    optimizer.zero_grad()
    
    # Speichere Loss
    loss_hist.append(loss.item())
    
    # Evaluierung
    if it % 100 == 0:
        # Loss auf Testdaten berechnen
        with torch.no_grad():
            y_pred_test = net(x_test)
            print(loss_fn(y_pred_test, y_test))

        # Ergebnisse grafisch darstellen
        plt.plot(x_test, y_test, '.')
        plt.plot(x_test, y_pred_test, '.')
        plt.legend(['Testdaten', 'Vorhersage NN'])
        plt.show()

In [ ]:
# Loss während des Trainings plotten
plt.plot(loss_hist)
plt.show()

In [ ]:
# Loss auf Testdaten berechnen
with torch.no_grad():
    y_pred_test = net(x_test)
    print(loss_fn(y_pred_test, y_test))

# Ergebnisse grafisch darstellen
plt.plot(x_test, y_test, '.')
plt.plot(x_test, y_pred_test, '.')
plt.legend(['Testdaten', 'Vorhersage NN'])
plt.show()

## Ziffern klassifizieren

### Aufgabe 7

Als nächstes widmen wir uns der Klassifikation von Ziffern. Wir tuen dies anhand des MNIST-Datensatzes, den man sich über die Funktion [`torchvision.datasets.MNIST`](https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST) herunterladen kann. Außerdem kannst du mit der Funktion direkt Transformationen auf dem Datensatz ausführen. Wir wollen die Bilder direkt mit [`torchvision.transforms.ToTensor`](https://pytorch.org/vision/stable/generated/torchvision.transforms.ToTensor.html#torchvision.transforms.ToTensor) zu Tensoren konvertieren.

Lade das Trainings- und Testset herunter und visualisiere einige der Ziffern.

In [ ]:
import torchvision

mnist_train = torchvision.datasets.MNIST('data/', train=True, 
                                         transform=torchvision.transforms.ToTensor(),
                                         download=True)
mnist_test = torchvision.datasets.MNIST('data/', train=False, 
                                         transform=torchvision.transforms.ToTensor(),
                                         download=True)

In [ ]:
for i in range(20):
    plt.imshow(torch.reshape(mnist_train[i][0], (28, 28)))
    plt.show()

    print(mnist_train[i][1])

### Aufgabe 8

Beim Training des letzten Modells haben wir die Mini-Batches manuell erzeugt. Allerdings gibt es die Funktion [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), die einem diese Aufgabe abnehmen.

Erzeuge einen solchen `DataLoader` für das Trainings- und Testset. Achte darauf, dass die Trainingsdaten gemischt werden müssen.

In [ ]:
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=16)

### Aufgabe 9

Der Aufbau der neuronalen Netzes, welches wir zur Klassifikaiton der Ziffern nehmen, ist ähnlich dem im vorherigen Problem, allerdings müssen wir einige Dinge beachten.

Da es sich um Bilder handelt, müssen wir diese erst in Vektoren umwandeln. Das ist mit der Funktion [`torch.nn.Flatten`](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) möglich. Außerdem muss das Netzwerk jetzt 10 Zahlen zurück geben, aus denen die Wahrscheinlichkeiten für die jeweiligen Klassen bestimmt werden können.

Implementiere ein solches Netz. Erstelle außerdem den `Optimizer` und den für Klassifikation benötigten [`torch.nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).

In [ ]:
net = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(28 ** 2, 64), 
    torch.nn.ReLU(), 
    torch.nn.Linear(64, 64), 
    torch.nn.ReLU(), 
    torch.nn.Linear(64, 64), 
    torch.nn.ReLU(), 
    torch.nn.Linear(64, 10))

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())

### Aufgabe 10

Um das Netzwerk zu trainieren, führen wir eine `for`-Schleife über den `DataLoader` aus. Damit iterieren wir einmal durch den gesamten Datensatz, was als eine Epoche bezeichnet wird. Mit einer zweiten `for`-Schleife können wir mehrere solcher Epochen ausführen.

Ergänze den Trainingsalgorithmus und trainiere dein Modell. Plotte anschließen den Loss.

In [ ]:
n_epoch = 5

loss_hist = []

for ep in range(n_epoch):
    for x_batch, y_batch in tqdm(train_loader):
        # Vorhersage des Netzwerks
        y_pred = net(x_batch)

        # Loss berechnen
        loss = loss_fn(y_pred, y_batch)

        # Backpropagation
        loss.backward()

        # Optimizer step
        optimizer.step()
        optimizer.zero_grad()

        # Speichere Loss
        loss_hist.append(loss.item())

In [ ]:
plt.plot(loss_hist)
plt.show()

### Aufgabe 11

Berechne die Genauigkeit deines Netzwerkes, also wie viele Bilder richtig klassifiziert werden, auf dem Testset.

In [ ]:
sum_correct = 0
sum_imgs = 0

for x_batch, y_batch in tqdm(test_loader):
    # Vorhersage des Netzes ohne Gradientenberechnung
    with torch.no_grad():
        y_pred = # ???
    
    # Vorhergesagtes Label
    y_pred = # ???
        
    # Anzahl der Bilder updaten
    sum_imgs += len(x_batch)
    
    # Anzahl der korrekt klassifizierten Bilder
    sum_correct += # ???

# Accuracy berechnen und ausgeben
accuracy = sum_correct / sum_imgs
print('Accuracy auf dem Testset: ', accuracy)



### Aufgabe 12

Wiederhole das Training und berechne die Genauigkeit diesmal nach jeder Epoche. Verbessere außerdem deine Netzwerkarchitektur.

In [ ]:
# Du kannst deinen Code aus den vorherigen Aufgaben nutzen und anpassen.